In [1]:
import numpy as np
import os
import tqdm
import hack_model
import helper
import torch

%load_ext autoreload
%autoreload 2

1. Load the HACK Model

In [2]:
device = torch.device("cuda")
hack = hack_model.HACK().to(device)
S = hack_model.load_pca(r"model/S.npy").to(device)
E = hack_model.load_pca(r"model/E.npy").to(device)
P = hack_model.load_pca(r"model/P.npy").to(device)

template_path = "model/000_generic_neutral_mesh.obj"
output = "output"


2. Generate a random sample

In [3]:
beta = torch.randn(1, 200).to(device)
theta = torch.zeros(1, 8, 3).to(device)
tau = torch.zeros(1, 1).to(device)
alpha = torch.ones(1, 1).to(device)
bsw = torch.zeros(1, 55).to(device)

T_transformed_0 = hack(theta, tau, alpha, bsw, T=S(beta), E=E(), P=P())["T_transformed"][0].cpu().numpy()
os.makedirs(output, exist_ok=True)
helper.write_obj(template_path, T_transformed_0, f"{output}/T_transformed_0.obj")


3. Generate a pre-recorded sequence

In [4]:
theta_list = torch.tensor(np.load(r"data/theta_list.npy")).to(torch.float32).to(device)
tau_list = torch.tensor(np.load(r"data/tau_list.npy")).to(torch.float32).to(device)
alpha_list = torch.tensor(np.load(r"data/alpha_list.npy")).to(torch.float32).to(device)
bsw_list = torch.tensor(np.load(r"data/bsw_list.npy")).to(torch.float32).to(device)

os.makedirs(f"{output}/seq", exist_ok=True)

for i, (theta, tau, alpha, bsw) in tqdm.tqdm(enumerate(zip(theta_list, tau_list, alpha_list, bsw_list)), total=len(theta_list)):
    T_transformed_0 = hack(theta[None], tau[None], alpha[None], bsw[None], T=S(beta), E=E(), P=P())["T_transformed"][0].cpu().numpy()
    helper.write_obj(template_path, T_transformed_0, f"{output}/seq/{i:06d}.obj")


100%|██████████| 500/500 [00:15<00:00, 32.60it/s]
